In [26]:
#poetry install python-dotenv
import pandas as pd
from sodapy import Socrata
import os
from dotenv import load_dotenv
import re

load_dotenv('.env')
#TOKEN_APP = os.getenv("SECRET_TOKEN")
#TOKEN_USER = os.getenv("TOKEN_USER")
#TOKEN_PASS = os.getenv("TOKE_PASS")

client = Socrata(
        "www.datos.gov.co", None)
        #TOKEN_APP,
        #username= TOKEN_USER,
        #password= TOKEN_PASS,
        #timeout=5000)


consulta =  """SELECT 
proveedor_adjudicado,
tipo_de_contrato, 
modalidad_de_contratacion,
valor_del_contrato,
fecha_de_firma,
estado_contrato, 
duraci_n_del_contrato,
condiciones_de_entrega 	
WHERE nombre_entidad = "FABRICA DE LICORES DEL TOLIMA"
limit 2000"""
data_id = "jbjy-vk9h"

results = client.get(data_id, query = consulta)
results_df = pd.DataFrame.from_records(results)

In [28]:
results_df.head()


,proveedor_adjudicado,tipo_de_contrato,modalidad_de_contratacion,valor_del_contrato,fecha_de_firma,estado_contrato,duraci_n_del_contrato,condiciones_de_entrega,duracion_en_dias
0,CASTILLO GRUPO EMPRESARIAL,Suministros,Contratación directa,20050000,2024-03-11T00:00:00.000,En ejecución,4 Dia(s),No Definido,4.0
1,MEGATEX DOTACIONES SAS,Suministros,Contratación directa,162542100,2023-02-14T00:00:00.000,En ejecución,11 Mes(es),No Definido,330.0
2,KAREN CAROLAY GUTIERREZ ORTIZ,Prestación de servicios,Contratación directa,2550000,2021-11-09T00:00:00.000,terminado,51 Dia(s),A convenir,51.0
3,PEDRO JOSE CASTILLO JARAMILLO,Prestación de servicios,Contratación directa,32000000,2023-05-05T00:00:00.000,En ejecución,8 Mes(es),A convenir,240.0
4,Johnnatan Pulgarin aguirre,Prestación de servicios,Contratación directa,5000000,2023-05-19T00:00:00.000,En ejecución,2 Mes(es),No Definido,60.0


In [27]:

# Función para convertir el tiempo a días
def convertir_a_dias(tiempo):
    match = re.match(r'(\d+)\s*(Dia\(s\)|Mes\(es\))', tiempo)
    if match:
        cantidad = int(match.group(1))
        unidad = match.group(2)
        if unidad == 'Dia(s)':
            return cantidad
        elif unidad == 'Mes(es)':
            return cantidad * 30  # Asumiendo 30 días por mes
    return None
 
# Aplicar la función a la columna 'duracion'
results_df['duracion_en_dias'] = results_df['duraci_n_del_contrato'].apply(convertir_a_dias)

In [29]:
results_df.proveedor_adjudicado = results_df.proveedor_adjudicado.str.upper()

In [30]:
results_df.proveedor_adjudicado.value_counts().head(10)

proveedor_adjudicado
SIN DESCRIPCION                      50
LEON GRAFICAS S.A.S                  13
ALLARTE LOGISTICA EMPRESARIAL SAS    13
SANDRA MILENA GUTIERREZ BUITRAGO     11
EDWIN ANDRÉS CASTRO NAVARRO          10
CARACOL S.A.                         10
NATALIA ANDREA DUQUE MUÑOZ           10
JOSE FERNANDO PASSOS VILLARREAL      10
RUTH MARINA ZEA JAIQUEL               9
RCN RADIO S.A.S                       9
Name: count, dtype: int64

In [31]:
results_df.tipo_de_contrato.value_counts()

tipo_de_contrato
Prestación de servicios       1016
Suministros                     72
Compraventa                     24
Consultoría                      4
Otro                             4
Seguros                          3
Arrendamiento de inmuebles       2
Servicios financieros            1
Venta muebles                    1
Obra                             1
Name: count, dtype: int64

In [9]:
results_df.nunique()

tipo_de_contrato              10
nombre_entidad                 1
fecha_de_firma               355
estado_contrato                9
modalidad_de_contratacion      7
condiciones_de_entrega         5
valor_del_contrato           416
duraci_n_del_contrato         95
proveedor_adjudicado         444
dtype: int64